In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import PipelineModel
from pyspark.sql.functions import col
from pyspark.sql import Row
from pyspark.sql.types import *
from functools import reduce
import numpy as np
import time
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
# Only works for Jupyter Notebooks!
%matplotlib inline

In [2]:
spark = SparkSession \
    .builder \
    .appName("Tweet Streaming App") \
    .getOrCreate()



In [3]:
socketDF = spark \
    .readStream \
    .format("socket") \
    .option("host", "127.0.0.1") \
    .option("port", 5559) \
    .load()

In [4]:
# socketDF.isStreaming()
socketDF.printSchema()


root
 |-- value: string (nullable = true)



In [5]:
oldColumns = socketDF.schema.names
# oldColumns.for
newColumns = ["text"]
df = reduce(lambda socketDF, idx: socketDF.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), socketDF)

In [6]:
rf = PipelineModel.load("./tmp/rasNas/")
prediction = rf.transform(df)

In [58]:
File_save2 = prediction.writeStream.format("console").start()

In [56]:
# File_save2.awaitTermination()
File_save2.stop()

In [7]:
sentTweets = prediction.registerTempTable("tweets")

In [48]:
# spark.sql("select text from tweets").writeStream.format("console").start()
sentTweets = spark.sql("select prediction from tweets")
spark.stop()

In [7]:
# # sentTweetsDF = sentTweets.toPandas()
# count =0
# while count < 10:
    
# #     time.sleep( 3 )
# #     top_10_tweets = sq.sql( 'Select prediction from tweets' )
# #     display.clear_output(wait=True)
# #     plt.figure( figsize = ( 10, 8 ) )
# #     sns.tsplot(data=sentTweets.select("prediction"))
#     fig, ax = plt.subplots()
#     ax.plot(np.array( sentTweets.select('prediction')[0]))
#     plt.show()
#     count = count + 1